Import the libraries, as always, and read in the data

In [ ]:
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_constants import GALYLEO_STRING, GALYLEO_NUMBER
from galyleo.galyleo_jupyterlab_client import GalyleoClient
import csv
def cleanse_row(row):
    values = [entry.strip() for entry in row]
    return [int(values[i]) for i in range(3) ] + values[3:7] + [float(values[7])]
with open('../ufos.csv', 'r') as ufo_file:
    reader = csv.reader(ufo_file)
    next(reader)
    data = [cleanse_row(row) for row in reader]
ufo_file.close()

Let's aggregate by country and year, ignoring all the other fields.  First, find the unique countries and years

In [ ]:
countries = set([row[4] for row in data])
years = set([row[2] for row in data])
keys = [(country, year) for country in countries for year in years]

Now let's count

In [ ]:
count = {}
for key in keys: count[key] = 0
for row in data:
    count[(row[4], row[2])] = count[(row[4], row[2])] + 1

Now let's send our counts to the dashboard.  WARNING! WARNING!  Make sure you open a dashboard (File>New>Galyleo Dashboard)

In [ ]:
table = GalyleoTable('sightings_year_country')
schema = [('Country',  GALYLEO_STRING), ('Year', GALYLEO_NUMBER), ('Count',  GALYLEO_NUMBER)]
table_data = [[key[0], key[1], count[key]] for key in keys]
table.load_from_schema_and_data(schema, table_data)

LAST CHANCE!!! IF YOU HAVEN'T ALREADY, OPEN UP A DASHBOARD NOW!!!

In [ ]:
client = GalyleoClient()
client.send_data_to_dashboard(table)

Click over to your new dashboard, open up the sidebar flap with a click on the button (middle, far right side) and observe that under the list on the Tables tab there is an entry, sightings_year_country.  The data was sent!  Now, let's do a selective send.  We're going to open another dashboard, so we have two open at once.  Rename the new dashboard to "test.gd.json".
Now aggregate by count and state

In [ ]:
state_records = [row for row in data if len(row[3]) > 0]
states = set([row[3] for row in state_records])
new_keys = [(state, year) for state in states for year in years]
state_count = {}
for key in new_keys: count[key] = 0
for row in state_records:
    count[(row[3], row[2])] = count[(row[3], row[2])] + 1
table2 = GalyleoTable('sightings_year_state')
schema2 = [('State',  GALYLEO_STRING), ('Year', GALYLEO_NUMBER), ('Count',  GALYLEO_NUMBER)]
table2_data = [[key[0], key[1], count[key]] for key in keys]
table2.load_from_schema_and_data(schema2, table2_data)


Send the new table _only_ to "test.gd.json"

In [ ]:
client.send_data_to_dashboard(table2, 'test1.gd.json')